In [1]:
import torch
import torch.nn as nn
import numpy as np
from source.featlearn.autoencoder_lr import AutoencoderFL


/home/texs/anaconda3/envs/clearn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
length = 28
channels = 5
x =  torch.zeros([32, length, channels])
ae = CNN_AE( channels, length, 5, out_channels=128, backbone=True)

28
0
0
4


In [25]:
enc = ae(x)
print(enc.shape)
# 364 % 4 != 0

torch.Size([32, 5, 365])
torch.Size([32, 32, 182])
torch.Size([32, 64, 91])
torch.Size([32, 128, 45])
torch.Size([32, 64, 90])
True
False
True
Paddinbg?
torch.Size([32, 64, 91])
Hereeeeeee?
torch.Size([32, 32, 182])
Here?
Paddinbg?
torch.Size([32, 5, 365])
torch.Size([32, 5, 365])
torch.Size([32, 5760])
